<a href="https://colab.research.google.com/github/Karla-Flores/Project-04/blob/main/Chicago_Crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Preprocessing**

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving chicago_crime_db.csv to chicago_crime_db (1).csv


In [3]:
# Uploading ETL file
data = pd.read_csv('chicago_crime_db.csv')
data.head(10)

,Primary Type,Arrest,Domestic,Latitude,Longitude,Location Description,HOUR,SEASON
0,BATTERY,False,True,41.815117,-87.670000,RESIDENCE,Afternoon,Autumn
1,THEFT,False,False,41.895080,-87.765400,CTA,Morning,Autumn
2,NARCOTICS,True,False,41.937406,-87.716650,PUBLIC OPEN SPACE,Afternoon,Autumn
3,ASSAULT,False,True,41.881903,-87.755121,RESIDENCE,Afternoon,Autumn
4,BURGLARY,False,False,41.744379,-87.658431,RESIDENCE,Morning,Autumn
5,BURGLARY,False,False,41.914636,-87.681631,RESIDENCE,Evening,Autumn
6,THEFT,True,False,41.851989,-87.689219,COMMERCIAL BUILDING,Afternoon,Autumn
7,ROBBERY,False,True,41.882814,-87.704326,PUBLIC OPEN SPACE,Morning,Autumn
8,THEFT,False,False,41.763648,-87.722345,PUBLIC OPEN SPACE,Afternoon,Autumn
9,BATTERY,False,False,41.975968,-87.768014,COMMERCIAL BUILDING,Morning,Autumn


In [4]:
data.shape

(6691918, 8)

In [5]:
data.loc[data['Primary Type'] == 'CRIMINAL DAMAGE', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'CRIMINAL TRESPASS', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'DECEPTIVE PRACTICE', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'INTERFERENCE WITH PUBLIC OFFICER', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'LIQUOR LAW VIOLATION', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'PUBLIC PEACE VIOLATION', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'WEAPONS VIOLATION', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'
data.loc[data['Primary Type'] == 'GAMBLING', 'Primary Type'] = 'CRIMINAL LAW VIOLATION'

In [6]:
data.loc[data['Primary Type'] == 'OTHER OFFENSE', 'Primary Type'] = 'OTHER'

In [7]:
data.loc[data['Primary Type'] == 'BATTERY', 'Primary Type'] = 'ASSAULT'

In [8]:
data.loc[data['Primary Type'] == 'STALKING', 'Primary Type'] = 'INTIMIDATION'

In [9]:
data.loc[data['Primary Type'] == 'BURGLARY', 'Primary Type'] = 'ROBBERY'
data.loc[data['Primary Type'] == 'MOTOR VEHICLE THEFT', 'Primary Type'] = 'ROBBERY'
data.loc[data['Primary Type'] == 'THEFT', 'Primary Type'] = 'ROBBERY'

In [10]:
data.loc[data['Primary Type'] == 'CRIM SEXUAL ASSAULT', 'Primary Type'] = 'SEX OFFENSE'
data.loc[data['Primary Type'] == 'CRIMINAL SEXUAL ASSAUL', 'Primary Type'] = 'SEX OFFENSE'
data.loc[data['Primary Type'] == 'PROSTITUTION', 'Primary Type'] = 'SEX OFFENSE'

In [11]:
sample = data.sample(frac=0.05)

In [12]:
sample.shape

(334596, 8)

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
sample_dummies = pd.get_dummies(sample, columns = ['Location Description', 'HOUR', 'SEASON'],dtype=float)
sample_dummies.head()

,Primary Type,Arrest,Domestic,Latitude,Longitude,Location Description_AIRPORT,Location Description_CHURCH,Location Description_COMMERCIAL BUILDING,Location Description_CTA,Location Description_EDUCATIONAL BUILDING,Location Description_FEDERAL PROPERTY,Location Description_HOSPITAL,Location Description_HOTEL,Location Description_OTHER,Location Description_PUBLIC ENTERTAINMENT,Location Description_PUBLIC OPEN SPACE,Location Description_RESIDENCE,Location Description_VEHICLE,HOUR_Afternoon,HOUR_Evening,HOUR_Morning,HOUR_Night,SEASON_Autumn,SEASON_Spring,SEASON_Summer,SEASON_Winter
4881434,ROBBERY,False,False,41.839816,-87.617516,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1374799,CRIMINAL LAW VIOLATION,False,False,41.786320,-87.654673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1364844,SEX OFFENSE,True,False,41.880519,-87.743442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6385174,ROBBERY,False,False,41.807910,-87.677102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3962036,ROBBERY,False,False,41.913615,-87.670215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
# Split our preprocessed data into our features and target arrays
y = sample_dummies['Primary Type']
X = sample_dummies.drop('Primary Type', axis=1)

In [37]:
len(y.unique())

12

In [15]:
X

,Arrest,Domestic,Latitude,Longitude,Location Description_AIRPORT,Location Description_CHURCH,Location Description_COMMERCIAL BUILDING,Location Description_CTA,Location Description_EDUCATIONAL BUILDING,Location Description_FEDERAL PROPERTY,Location Description_HOSPITAL,Location Description_HOTEL,Location Description_OTHER,Location Description_PUBLIC ENTERTAINMENT,Location Description_PUBLIC OPEN SPACE,Location Description_RESIDENCE,Location Description_VEHICLE,HOUR_Afternoon,HOUR_Evening,HOUR_Morning,HOUR_Night,SEASON_Autumn,SEASON_Spring,SEASON_Summer,SEASON_Winter
4881434,False,False,41.839816,-87.617516,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1374799,False,False,41.786320,-87.654673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1364844,True,False,41.880519,-87.743442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6385174,False,False,41.807910,-87.677102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3962036,False,False,41.913615,-87.670215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976193,False,False,41.909510,-87.755034,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5507828,True,True,41.854658,-87.708992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2176461,False,True,41.894315,-87.717501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6054245,False,False,41.917787,-87.732697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
len(y.unique())

12

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
list(le.classes_)
y_label = le.transform(y)
y_label

array([10,  2, 11, ...,  1, 10, 11])

In [18]:
len(y_label)

334596

In [19]:
y_label.shape

(334596,)

In [38]:
from tensorflow.keras.utils import to_categorical


In [40]:
Y = to_categorical(y_label, 12)
Y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [41]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state= 42)

In [42]:
# Standardizing dataset so columns that contain larger values do not unduly influence the outcome
scaler = StandardScaler().fit(X)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
X_train_scaled.shape

(250947, 25)

In [45]:
y_train.shape

(250947, 12)

# Neural Network


In [62]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# # First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=25))

# # Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# # Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# # Output layer
nn.add(tf.keras.layers.Dense(units=12, activation="softmax"))

# # Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 100)               2600      
_________________________________________________________________
dense_25 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_26 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_27 (Dense)             (None, 12)                372       
Total params: 9,552
Trainable params: 9,552
Non-trainable params: 0
_________________________________________________________________


In [63]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [64]:
y_train.shape

(250947, 12)

In [65]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2453 - accuracy: 0.5295
Epoch 2/100
7843/7843 [==============================] - 14s 2ms/step - loss: 1.2247 - accuracy: 0.5334
Epoch 3/100
7843/7843 [==============================] - 14s 2ms/step - loss: 1.2189 - accuracy: 0.5345
Epoch 4/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2155 - accuracy: 0.5353
Epoch 5/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2135 - accuracy: 0.5351
Epoch 6/100
7843/7843 [==============================] - 16s 2ms/step - loss: 1.2117 - accuracy: 0.5359
Epoch 7/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2106 - accuracy: 0.5356
Epoch 8/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2092 - accuracy: 0.5364
Epoch 9/100
7843/7843 [==============================] - 15s 2ms/step - loss: 1.2083 - accuracy: 0.5364
Epoch 10/100
7843/7843 [==============================] - 15s 2m

In [66]:
# Evaluate the model using the training data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2615/2615 - 3s - loss: 1.2248 - accuracy: 0.5355
Loss: 1.2248083353042603, Accuracy: 0.5354875922203064
